# English to french 

## using sequence to sequence learning model

# IMPORTING LIBRARIES

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow import keras

In [3]:
from keras.models import Model
from keras.layers import LSTM, Dense, Input

In [4]:
batch_size=64
epochs=100
latent_dim=256
num_samples=10000
data_path=r'C:\Users\Lenovo\Desktop\fake news\translate\fra.txt'

In [5]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines=f.read().split('\n')
for line in lines[: min(num_samples, len(lines)-1)]:
    input_text,target_text,_= line.split('\t')
    target_text='\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
lines[0].split('\t')

['Go.',
 'Va !',
 'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)']

In [7]:
'\t'+lines[0].split('\t')[1]

'\tVa !'

In [8]:
input_characters

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é'}

In [9]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [10]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length inputs:', max_encoder_seq_length)
print('Max sequence length outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length inputs: 15
Max sequence length outputs: 59


In [11]:
input_token_index=dict([char,i] for i,char in enumerate(input_characters))
target_token_index=dict([char,i] for i,char in enumerate(target_characters))

In [12]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [13]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '0': 12,
 '1': 13,
 '2': 14,
 '3': 15,
 '5': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 '\xa0': 71,
 '«': 72,
 '»': 73,
 'À': 74,
 'Ç': 75,
 'É': 76,
 'Ê': 77,
 'à': 78,
 'â': 79,
 'ç': 80,
 'è': 81,
 'é': 82,
 'ê': 83,
 'î': 84,
 'ï': 85,
 'ô': 86,
 'ù': 87,
 'û': 88,
 'œ': 89,
 '\u2009': 90,
 '’': 91,
 '\u202f': 92}

In [14]:
encoder_input_data=np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [20]:
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t, input_token_index[char]]=1
    encoder_input_data[i,t+1:,input_token_index[' ']]=1
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]]=1
        if t>0:
            decoder_target_data[i,t-1,target_token_index[char]]=1
    decoder_input_data[i,t+1:, target_token_index[' ']]=1
    decoder_target_data[i,t:,target_token_index[' ']]=1

In [21]:
encoder_input_data[0].shape

(15, 71)

## Create LSTM layer

Encoding

In [22]:
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c=encoder(encoder_inputs)
encoder_states=[state_h, state_c]

In [23]:
decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dim, return_sequences=True ,return_state=True)
decoder_outputs, _,_=decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [26]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [27]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,
          epochs=epochs,validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 22s 158ms/step - loss: 1.1484 - accuracy: 0.7354 - val_loss: 1.0356 - val_accuracy: 0.7165
Epoch 2/100
125/125 [==============================] - 19s 152ms/step - loss: 0.8289 - accuracy: 0.7772 - val_loss: 0.8226 - val_accuracy: 0.7780
Epoch 3/100
125/125 [==============================] - 19s 149ms/step - loss: 0.6566 - accuracy: 0.8154 - val_loss: 0.6924 - val_accuracy: 0.8010
Epoch 4/100
125/125 [==============================] - 19s 154ms/step - loss: 0.5719 - accuracy: 0.8340 - val_loss: 0.6189 - val_accuracy: 0.8181
Epoch 5/100
125/125 [==============================] - 19s 156ms/step - loss: 0.5233 - accuracy: 0.8468 - val_loss: 0.6023 - val_accuracy: 0.8270
Epoch 6/100
125/125 [==============================] - 21s 167ms/step - loss: 0.4901 - accuracy: 0.8559 - val_loss: 0.5555 - val_accuracy: 0.8382
Epoch 7/100
125/125 [==============================] - 19s 156ms/step - loss: 0.4629 - accuracy: 0.8636 - val_loss: 0.5500 -